In [1]:
import math
import os
import requests
from collections import defaultdict
from tqdm import tqdm
import hashlib
import numpy as np
import json
from haversine import haversine


def deg2num(lat_deg, lon_deg, zoom):
    lat_rad = math.radians(lat_deg)
    n = 2.0 ** zoom
    xtile = int((lon_deg + 180.0) / 360.0 * n)
    ytile = int((1.0 - math.log(math.tan(lat_rad) + 1
                / math.cos(lat_rad)) / math.pi) / 2.0 * n)
    return (xtile, ytile)


def deg2num(lat_deg, lon_deg, zoom):
    lat_rad = math.radians(lat_deg)
    n = 2.0 ** zoom
    xtile = int((lon_deg + 180.0) / 360.0 * n)
    ytile = int((1.0 - math.log(math.tan(lat_rad) + 1
                / math.cos(lat_rad)) / math.pi) / 2.0 * n)
    return (xtile, ytile)


def num2deg(xtile, ytile, zoom):
    n = 2.0 ** zoom
    lon_deg = xtile / n * 360.0 - 180.0
    lat_rad = math.atan(math.sinh(math.pi * (1 - 2 * ytile / n)))
    lat_deg = math.degrees(lat_rad)
    return (lat_deg, lon_deg)


def get_work_queue(lats, lons, zoom):
    top_left = deg2num(max(lats), min(lons), zoom)
    bottom_right = deg2num(min(lats), max(lons), zoom)

    tiles = []
    for i in range(top_left[0], bottom_right[0]):
        for j in range(top_left[1], bottom_right[1]):
            tiles.append((i, j + 1))
    return tiles

## Code that determines which nodes are 'useful'

In [2]:
def get_tile_edges(tile_x, tile_y, zoom):
    (north, west) = num2deg(tile_x, tile_y, zoom)
    (south, east) = num2deg(tile_x + 1, tile_y + 1, zoom)
    return (east, north, west, south)


def get_edge_nodes(nodes, bounds):
    (e, n, w, s) = bounds
    oob = set()
    for node in nodes.values():
        lat = node['geo:lat']
        long = node['geo:long']
        if not s <= lat <= n or not w <= long <= e:
            oob.add(node['@id'])
    return oob


def calculate_node_degree(ways):
    result = defaultdict(int)
    for way in ways.values():
        for node_id in way['osm:hasNodes']:
            result[node_id] += 1
    return result


def get_useful_nodes(nodes, ways, bounds):
    result = set()
    degrees = calculate_node_degree(ways)
    oob = get_edge_nodes(nodes, bounds)  # nodes that are out of bounds

    a = b = c = d = e = f = 0

    for way in ways.values():
        way_nodes = way['osm:hasNodes']
        first = way_nodes[0]
        last = way_nodes[-1]

        for (i, node_id) in enumerate(way_nodes):
            node = nodes[node_id]
            if node_id in oob:
                a += 1
                result.add(node_id)
            elif degrees[node_id] > 1:
                b += 1
                result.add(node_id)
            elif i > 0 and way_nodes[i - 1] in oob:
                c += 1
                result.add(node_id)
            elif i < len(way_nodes) - 1 and way_nodes[i + 1] in oob:
                d += 1
                result.add(node_id)
            elif i == 0:
                e += 1
                result.add(node_id)
            elif i == len(way_nodes) - 1:
                f += 1
                result.add(node_id)
            elif i == 1 and first in oob:

                # this will be the first out of bounds node in another tile

                result.add(node_id)
            elif i == len(way_nodes) - 2 and last in oob:

                # this will be the first out of bounds node in another tile

                result.add(node_id)
            elif node.get('osm:highway') or node.get('osm:barrier') \
                or node.get('osm:crossing'):

                # important information for routing

                result.add(node_id)
    return result

## The actual reduction code

In [3]:
def get_distance(nodes, node1, node2):
    node1 = nodes[node1]
    node2 = nodes[node2]

    return int(round(haversine((node1['geo:lat'], node1['geo:long']),
               (node2['geo:lat'], node2['geo:long']), unit='m')))


def reduce_way(way, all_nodes, useful_nodes):
    nodes = []
    distances = []

    way_nodes = way['osm:hasNodes']
    last_useful_node = way_nodes[0]
    nodes.append(last_useful_node)
    last_node = last_useful_node
    distance_since = 0

    for node_id in way_nodes[1:]:
        distance_since += get_distance(all_nodes, node_id, last_node)
        if node_id in useful_nodes:
            nodes.append(node_id)
            distances.append(distance_since)
            last_useful_node = node_id
            distance_since = 0
        last_node = node_id
    return (nodes, distances)


def reduce_tile(tile_x, tile_y, zoom):
    reduced_file_name = get_reduced_file_name(tile_x, tile_y, zoom)
    if True or not os.path.isfile(reduced_file_name):
        base_file_name = get_file_name(tile_x, tile_y, zoom)
        (nodes, ways) = get_tile(base_file_name)
        bounds = get_tile_edges(tile_x, tile_y, zoom)
        useful_nodes = get_useful_nodes(nodes, ways, bounds)

        for way in ways.values():
            (reduced_nodes, weights) = reduce_way(way, nodes,
                    useful_nodes)
            way.pop('osm:hasNodes')
            way['osm:hasEdges'] = {'osm:hasNodes': reduced_nodes,
                                   'osm:hasWeights': weights}

        nodes = dict((id, node) for (id, node) in nodes.items() if id
                     in useful_nodes)
        write_to_file(tile_x, tile_y, zoom, nodes, ways)

## IO code

In [4]:
class Element(dict):

    def __init__(self, values):
        super().__init__(values)
        self.__dict__.update(values)


def get_tile(filename):
    nodes = {}
    ways = {}

    try:
        with open(filename) as f:
            obj = json.load(f)
            graph = obj['@graph']
            for element in graph:
                e = Element(element)
                e.id = e['@id']
                if element.get('geo:long'):
                    e.lat = e['geo:lat']
                    e.long = e['geo:long']
                    nodes[element['@id']] = e
                if element.get('osm:hasNodes'):
                    ways[element['@id']] = Element(element)
    except Exception as e:
        pass

    return (nodes, ways)


def write_to_file(
    tile_x,
    tile_y,
    zoom,
    nodes,
    ways,
    ):
    file_name = get_reduced_file_name(tile_x, tile_y, zoom)

    graph = list(nodes.values()) + list(ways.values())

    blob = {
        '@context': {
            'tiles': 'https://w3id.org/tree/terms#',
            'hydra': 'http://www.w3.org/ns/hydra/core#',
            'osm': 'https://w3id.org/openstreetmap/terms#',
            'rdfs': 'http://www.w3.org/2000/01/rdf-schema#',
            'geo': 'http://www.w3.org/2003/01/geo/wgs84_pos#',
            'dcterms': 'http://purl.org/dc/terms/',
            'dcterms:license': {'@type': '@id'},
            'hydra:variableRepresentation': {'@type': '@id'},
            'hydra:property': {'@type': '@id'},
            'osm:access': {'@type': '@id'},
            'osm:barrier': {'@type': '@id'},
            'osm:bicycle': {'@type': '@id'},
            'osm:construction': {'@type': '@id'},
            'osm:crossing': {'@type': '@id'},
            'osm:cycleway': {'@type': '@id'},
            'osm:footway': {'@type': '@id'},
            'osm:highway': {'@type': '@id'},
            'osm:motor_vehicle': {'@type': '@id'},
            'osm:motorcar': {'@type': '@id'},
            'osm:oneway_bicycle': {'@type': '@id'},
            'osm:oneway': {'@type': '@id'},
            'osm:smoothness': {'@type': '@id'},
            'osm:surface': {'@type': '@id'},
            'osm:tracktype': {'@type': '@id'},
            'osm:vehicle': {'@type': '@id'},
            'osm:hasNodes': {'@container': '@list', '@type': '@id'},
            'osm:hasMembers': {'@container': '@list', '@type': '@id'},
            },
        '@id': 'https://http://hdelva.be/tiles/reduced/{}/{}/{}'.format(zoom,
                tile_x, tile_y),
        'tiles:zoom': zoom,
        'tiles:longitudeTile': tile_x,
        'tiles:latitudeTile': tile_y,
        'dcterms:isPartOf': {
            '@id': 'https://tiles.openplanner.team/planet/',
            '@type': 'hydra:Collection',
            'dcterms:license': 'http://opendatacommons.org/licenses/odbl/1-0/',
            'dcterms:rights': 'http://www.openstreetmap.org/copyright',
            'hydra:search': {
                '@type': 'hydra:IriTemplate',
                'hydra:template': 'https://tiles.openplanner.team/planet/14/{x}/{y}',
                'hydra:variableRepresentation': 'hydra:BasicRepresentation',
                'hydra:mapping': [{
                    '@type': 'hydra:IriTemplateMapping',
                    'hydra:variable': 'x',
                    'hydra:property': 'tiles:longitudeTile',
                    'hydra:required': True,
                    }, {
                    '@type': 'hydra:IriTemplateMapping',
                    'hydra:variable': 'y',
                    'hydra:property': 'tiles:latitudeTile',
                    'hydra:required': True,
                    }],
                },
            },
        '@graph': graph,
        }

    with open(file_name, 'w') as f:
        json.dump(blob, f)

## Lines of code to tinker with to change the area/zoom level

In [5]:
def get_file_name(tile_x, tile_y, zoom):
    return 'tiles/tr_{}/{}/{}.json'.format(zoom, tile_x, tile_y)


def get_reduced_file_name(tile_x, tile_y, zoom):
    dir_name = 'tiles/in_{}/{}/'.format(zoom, tile_x)
    if not os.path.isdir(dir_name):
        os.mkdir(dir_name)
    return '{}/{}.json'.format(dir_name, tile_y)

In [ ]:
# bounding box of data that needs to be processed
lats = [49, 52]
lons = [2.25, 6.6]

# zoom level that needs to be created
zoom = 9  

for tile_x, tile_y in tqdm(get_work_queue(lats, lons, zoom)):
    reduce_tile(tile_x, tile_y, zoom)

 89%|████████▉ | 32/36 [00:12<00:01,  2.00it/s]